# PyTorch biblioteka

**PyTorch** je Python biblioteka koja je jedna od najkorišćenijih biblioteka za duboko učenje. Slično kao i biblioteka numpy, pytorch biblioteka se zasniva na operacijama nad višedimenzionim nizovima. Za razliku od biblioteke numpy, pytorch podrža autmatsko računanje gradijenata, kao i ubrzavanje izračunavanja korišćenjem grafičkih kartica, što značajno olakšava i ubrzava optimizaciju neuronskih mreža. Na [ovom](https://pytorch.org/) linku se nalazi zvanična stranica biblioteke, a [ovde](https://pytorch.org/tutorials/beginner/basics/intro.html) se može pronaći koristan tutorijal.

In [5]:
import torch

Višedimenzione nizove u biblioteci pytorch nazivamo tenzorima i operacije nad njima su vrlo slične operacijama iz biblioteke numpy. 

In [6]:
# pravljenje tenzora od niza
X = torch.tensor([[1, 2, 3], [4, 5, 6]])
print('X =', X)
print('X.shape =', X.shape)
print('X.dtype =', X.dtype)

print('----')
# promena tipa podataka u tenzoru
X = X.to(torch.float32)
# X = X.float()
print('X =', X)
print('X.dtype =', X.dtype)

print('----')
# pravljenje tenzora sa podacima iz normalne raspodele
Y = torch.randn_like(X)
# Y = torch.randn(X.shape())
print(Y)
print('Y.shape =', Y.shape)
print('Y.dtype =', Y.dtype)

print('----')
# aritmetičke operacije
print('X + Y =', X + Y)
print('X * Y =', X * Y)
print('X @ Y.T =', X @ Y.T)
print('2^X =', 2 ** X)
print('sin(X) =', torch.sin(X))

print('----')
print('X.sum() =', X.sum())
print('X.sum(dim=0) =', X.sum(dim=0))
print('X.sum(dim=1) =', X.sum(dim=1))

X = tensor([[1, 2, 3],
        [4, 5, 6]])
X.shape = torch.Size([2, 3])
X.dtype = torch.int64
----
X = tensor([[1., 2., 3.],
        [4., 5., 6.]])
X.dtype = torch.float32
----
tensor([[ 0.9992,  1.0344, -1.9470],
        [-0.2329, -1.4034, -0.5485]])
Y.shape = torch.Size([2, 3])
Y.dtype = torch.float32
----
X + Y = tensor([[1.9992, 3.0344, 1.0530],
        [3.7671, 3.5966, 5.4515]])
X * Y = tensor([[ 0.9992,  2.0687, -5.8410],
        [-0.9318, -7.0168, -3.2912]])
X @ Y.T = tensor([[ -2.7730,  -4.6853],
        [ -2.5132, -11.2399]])
2^X = tensor([[ 2.,  4.,  8.],
        [16., 32., 64.]])
sin(X) = tensor([[ 0.8415,  0.9093,  0.1411],
        [-0.7568, -0.9589, -0.2794]])
----
X.sum() = tensor(21.)
X.sum(dim=0) = tensor([5., 7., 9.])
X.sum(dim=1) = tensor([ 6., 15.])


## Automatsko diferenciranje

Automatsko diferenciranje nam je dostupno kroz paket `torch.autograd`. Prilikom računanja sa tenzorima, pytorch pamti graf izračunavanja što joj omogućava da primenom pravila o izvodu složene funkcije, kretanjem u nazad kroz graf, izračuna gradijente tenzora čiji je parametar `requiers_grad` postavljen na `True`.

In [7]:
X = torch.tensor(1.0, requires_grad=True)
Y = torch.tensor(2.0, requires_grad=True)
Z = torch.tensor(3.0)
Z = X * Y * Z
Z.backward() # računanje gradijenta
print('X.grad =', X.grad)
print('Y.grad =', Y.grad)
print('Z.grad =', Z.grad) # None
print('-------')
print('Višestruko računanje gradijenta:')
X = torch.tensor(1.0, requires_grad=True)
l = X**2
l.backward()
print('X.grad =', X.grad) # X.grad = 2.0
l = X**3
l.backward()
print('X.grad =', X.grad) # X.grad = 5.0
print('Gradovi se u tenzoru sabiraju!')


X.grad = tensor(6.)
Y.grad = tensor(3.)
Z.grad = None
-------
Višestruko računanje gradijenta:
X.grad = tensor(2.)
X.grad = tensor(5.)
Gradovi se u tenzoru sabiraju!


/tmp/ipykernel_15660/3410228716.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1724788960438/work/build/aten/src/ATen/core/TensorBody.h:489.)
  print('Z.grad =', Z.grad) # None


Možemo primetiti da se gradijenti sabiraju svaki put kada računamo izvod.

Gradijent funkcije srednje kvadratne greške linearne regresije nad slučajnim podacima se može izračunati na sledeći način

In [8]:
X = torch.randn(32, 8)
W = torch.randn(1, 8, requires_grad=True)
b = torch.randn(1, requires_grad=True)
Y = torch.randn(32, 1)
Z = W @ X.T + b
L = ((Y - Z) ** 2).mean()
L.backward()
print('W.grad.shape =', W.grad.shape)
print('b.grad.shape =', b.grad.shape)

W.grad.shape = torch.Size([1, 8])
b.grad.shape = torch.Size([1])


## Gradijentni spust

Uz automatsko diferenciranje, biblioteka PyTorch ima implementirane različite algoritme gradijentnog spusta. Svi oni se nalaze u okviru `torch.optim` paketa gde su implementirani koraci gradijentnog spusta.

Ovde možemo videti primer optimizacije funkcije $f(x, y) = |x| + |y|$ sa 5 koraka gradijentnog spusta sa stopom učenja 1.

In [9]:
def f(x, y):
    return x.abs() + y.abs()

LEARNING_RATE = 1
NUMBER_OF_ITERATIONS = 5

x = torch.tensor(2.5, requires_grad=True)
y = torch.tensor(4.0, requires_grad=True)

# pravljenje optimizatora
optimizer = torch.optim.SGD([x, y], lr=LEARNING_RATE)

for step in range(NUMBER_OF_ITERATIONS):
    l = f(x, y)

    optimizer.zero_grad() # postavljanje gradijenata na nulu
    l.backward() # računanje gradijenata
    optimizer.step() # optimizacija
    print('----')
    print('step =', step)
    print('x =', x.item(), 'y =', y.item())



----
step = 0
x = 1.5 y = 3.0
----
step = 1
x = 0.5 y = 2.0
----
step = 2
x = -0.5 y = 1.0
----
step = 3
x = 0.5 y = 0.0
----
step = 4
x = -0.5 y = 0.0


## Rad sa podacima

U biblioteci PyTorch posotje dve korinse klase za rad sa podacima, obe u okviru paketa `torch.utils.data`:

- `Dataset` - klasa koju koristimo za učitavanje podataka. Cilj ove klase je da se pomoću nje može dobiti jedna instanca iz skupa podataka. 
- `Datloader` - klasa koja iz skupa podataka predstavljenog `Dataset` klasom izdvaja delove nad kojim će se vršiti jedan korak stohastičnog gradijentnog spusta.

In [10]:
from torch.utils.data import DataLoader, Dataset, TensorDataset, IterableDataset

class RandomDataset_v1(Dataset):
    def __init__(self, size=1000):
        super().__init__()
        self.data = torch.randn(size, 2)
        self.labels = torch.randn(size, 1)
        self.size = size

    def __len__(self):
        return self.size
    
    def __getitem__(self, index):
        return self.data[index], self.labels[index]

dataset1 = RandomDataset_v1()

class RandomDataset_v2(IterableDataset):
    def __init__(self):
        super().__init__()
    
    def __iter__(self):
        for _ in range(1000):
            yield torch.randn(2), torch.randn(1)

    def __len__(self):
        return 1000
    
dataset2 = RandomDataset_v2()

X = torch.randn(1000, 2)
Y = torch.randn(1000, 1)
dataset3 = TensorDataset(X, Y)
        

dataloader = DataLoader(dataset1, batch_size=32, shuffle=True, num_workers=4)
# dataloader = DataLoader(dataset2, batch_size=32, num_workers=4) # shuffle ne radi za IterableDataset
# dataloader = DataLoader(dataset3, batch_size=32, shuffle=True, num_workers=4)
for data, labels in dataloader:
    print('data.shape =', data.shape)
    print('labels.shape =', labels.shape)
    break



data.shape = torch.Size([32, 2])
labels.shape = torch.Size([32, 1])


Više o paketu `torch.utils.data` možete pronaći u [zvaničnoj dokumentaciji](https://pytorch.org/docs/stable/data.html)

## Podrška za neuronske mreže

Velika većina funkcionalnosti koje će nam biti potrebne za definicije neuronskih mreže se nalaze u okviru paketa `torch.nn`. Na primer ukoliko želimo da izvršimo linearnu transformaciju atributa, za to možemo koristiti klasu `torch.nn.Linear`, a za računanje srednje kvadratne greške možemo koristiti klasu `nn.MSELoss`.

In [11]:
import torch.nn as nn

x = torch.randn(32, 8)
y = torch.randn(32, 1)
linear = nn.Linear(8, 1)

print(linear(x).shape)

loss = nn.MSELoss()
print('loss =', loss(linear(x), y))

torch.Size([32, 1])
loss = tensor(1.8713, grad_fn=<MseLossBackward0>)


### Definisanje modela

Definisanje modela u biblioteci PyTorch se najčešće radi nasleđivanjem klase `nn.Module` i implemenitranjem metode `forward` koja se poziva korišćenjem () sintakse. Pored klase  

In [12]:
class LinearRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, x):
        return self.linear(x)


x = torch.randn(32, 8)
y = torch.randn(32, 1)

model = LinearRegression(8, 1)
loss = nn.MSELoss()

print(model)
print(model(x).shape)
print(loss(model(x), y))

LinearRegression(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)
torch.Size([32, 1])
tensor(0.8649, grad_fn=<MseLossBackward0>)


### Funkcionalni pristup

Za svaku klasu koju koristimo za neko izračunavanje (kao što su `nn.Linear` ili `nn.MSELoss`) postoji funckija u okviru paketa `torch.nn.functional` koju možemo koristiti za to izračunavanje. Ovo može biti posebno korisno ako je potrebno raditi neke nestandardne operacije koje liče na ove već postojeće.

In [13]:
import torch.nn.functional as F

class LinearRegression(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))
    
    def forward(self, x):
        return F.linear(x, self.weights, self.bias)

x = torch.randn(32, 8)
y = torch.randn(32, 1)

model = LinearRegression(8, 1)

print(model)
print(model(x).shape)
print(F.mse_loss(model(x), y))

LinearRegression()
torch.Size([32, 1])
tensor(5.4862, grad_fn=<MseLossBackward0>)


### Rad sa grafičkim karticama

Većina operacija koje se koriste u mašinskom učenju mogu se efikasno paralelizovati korišćenjem grafičkih kartica. Sve biblioteke mašinkog učenja iz ovog razloga moraju imati podršku za korišćenje grafičkih kartica. U biblioteci PyTorch svi tenzori imaju atribut `device` koji naglašava da li se on nalazi u radnoj memoriji računara ili u radnoj memoriji grafičke kartice. Kako bi se neka operacija između dva tenzora izvršila oba tenzora moraju imati isti atribut `device` i u zavisnosti od toga gde se nalaze, operaciju će izvršiti grafička kartica, odnosno procesor. Podrazumevano svi tenzori imaju `device` postavljen na `cpu`, a metodom `.to('cuda')` možemo ga prebaciti na grafičku karitcu.

In [ ]:
torch.cuda.is_available() # test da li je grafička kartica dostupna
# torch.cuda.device_count() # broj grafičkih kartica

device = 'cuda' if torch.cuda.is_available() else 'cpu' 

X = torch.randn(32, 8).to(device)  # ukoliko je grafička kartica dostupna, tenzor će biti na njoj
Y = torch.randn(32, 1).to(device)  # ukoliko je grafička kartica dostupna, tenzor će biti na njoj
linear = nn.Linear(8, 1).to(device) # ukoliko je grafička kartica dostupna, težine modela će biti na njoj
print('X.device =', X.device)
print('Y.device =', Y.device)
print('linear.device =', linear.weight.device)
print('linear.weight.device =', linear.weight.device)
print('linear.bias.device =', linear.bias.device)
print('linear(X).device =', linear(X).device)